
---

### Reminder: This 📘 `Python` notebook needs can be run from VS Code with [these prerequisites](../PREREQS.md).

#### How to use this notebook: 

* Just read the text and scroll along until you run into code blocks.
* Code blocks have computer code inside them — hover over the block and you can run the code.
* Run the code by hitting the ▶️ "play" button to the left. If the code runs you'll see a ✔️. If not, you'll get a ❌.
* The output and status of the code block will appear just below itself — you need to scroll down further to see it.
* Sometimes a code block will ask you for input in a hard-to-notice dialog box 👆 at the top of your notebook window. 

---

# Bonus Recipe: 💬 MiniChatGPT Clone
## 🧑‍🍳 Cook a simple "ChatGPT" clone

The magic of chat using LLM AI is that it's extremely simple to implement. If you're still feeling unsure about semantic functions, catch up [here](https://learn.microsoft.com/en-us/semantic-kernel/howto/semanticfunctions). 

## Step 1: Let's get started by instantiating a 🔥 kernel

You've already set up your API key information, so this should be an easy ▶️ and you're good to go.

In [ ]:
!pip install semantic-kernel==0.2.7.dev0

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import AzureTextCompletion, OpenAITextCompletion

kernel = sk.Kernel()

useAzureOpenAI = False

# Configure AI service used by the kernel
if useAzureOpenAI:
    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", AzureTextCompletion(deployment, endpoint, api_key))
else:
    api_key, org_id = sk.openai_settings_from_dot_env()
    kernel.add_text_completion_service("dv", OpenAITextCompletion("text-davinci-003", api_key, org_id))

😱 **Get an error message?** The [first notebook](../s1e1-ez-starter-notebook/notebook.ipynb) walks you through this process so you should be all set. But if you're still stuck, go to https://aka.ms/sk/discord where we have realtime support.

## Step 2️: Use the `ChatSkill` 🧂 skill to build a chatting AI in no time

We will be using the `Chat` and `ChatPersona` semantic functions that are accessible from within the `ChatSkill` subdirectory of `skills`

```
🗂️ skills
│
└─── 📁 FunSkill
└─── 📁 ChatSkill
     |
     └─── 📂 Chat
     └─── 📂 ChatPersona
```

And with that, let's get the LLM AI to tell us talk to us.

### Step 2.1: Create a simple chat loop using a basic kind of 🥑 memory

You'll be surprised to see how easy it is to make AI chat happen with the semantic function `ChatSkill.Chat.` It's easy as `history += <the new chat exchange>`! Or in other words, create a prompt that is incrementally just your running conversation that gets fed into the prompt each turn of the chat. The LLM's job then simply becomes the task to auto-complete what should get said next. The context is a kind of persistent 🥑 memory that gradually increases the richness of the prompt as more conversation is fed into `history.`

In [ ]:
# Load the Skills Directory
skillsDirectory = "./skills/"

# Load the Chat function from the FunSkill
chatSkill = kernel.import_semantic_skill_from_directory(skillsDirectory, "ChatSkill")

myContext = kernel.create_new_context()
botPrompt = "AI: Hello. What's your name?"
history = "{botPrompt}\n"

numberOfRounds = 4
myContext["history"] = history

for i in range(numberOfRounds):
    try:
        # get input from the user and set the context variable
        print("👆 Enter text in the input cell above to chat with the bot. 👆\n")
        sk_input = input(f"{botPrompt} ({(i+1)} of {numberOfRounds})")
        myContext["input"] = sk_input

        # run the chat function
        myResult = await kernel.run_async(chatSkill["Chat"], input_context=myContext)

        # tack onto the history 👇 what's come back from the model
        # ********************************************************
        theNewChatExchange = f"Me: {sk_input}\nAI:{myResult}\n"
        history += theNewChatExchange
        myContext["history"] = history

        # ********************************************************
        # this way the new chat exchange gets passed into the next round

        # announce the number of rounds and the history
        print(f"Chat for {i+1} of {numberOfRounds} rounds with AI:\n{history}")

        # prepare to "prompt" the user with the bot's response
        botPrompt = f"AI: {myResult}"
    except:
        # if the user hits "Escape" we end the chat early
        print("AI: Thanks for the wonderful chat!")
        break

> ✅ Note that the chat appears in the output above as you enter text in the little textbox that appears at the top of VS Code. It will stop after you've gone for `numberOfRounds` — so increase the number of rounds you'd like to chat with YOUR bot! You made it yourself!

### Step 2.2: Create a chat loop using 🥑 memory and with a personality of your choice

The accrual of `history += theNewChatExchange` is a beautiful thing to admire. This is the basic mechanism whereby the chats you have with ChatGPT can feel so realistic. That's because the prompt is getting slightly longer and longer as you chat. The reason "it gets you" is because it hasn't forgotten what it's chatted with you in the past. It's the extra 🥑 "fat" that makes the interaction with the AI so much more appealing.

Yet another facet to consider is how you're able to change the personality of the bot so easily. We'll use the `ChatSkill.ChatPersonality` function to do so with its added `$personality` context variable.

In [ ]:
# Load the Skills Directory
skillsDirectory = "./skills/"

# Load the Chat function from the FunSkill
chatSkill = kernel.import_semantic_skill_from_directory(skillsDirectory, "ChatSkill")

myContext = kernel.create_new_context() 
botName = "AI"

# Choose a personality here 👇 ...
# *********************************************************************/
personality = "grumpy and extremely unhelpful"
# *********************************************************************/

botPrompt = f"AI: Hello. My responses will be {personality}."
history = f"{botPrompt}\n"

numberOfRounds = 4

myContext["history"] = history
myContext["personality"] = personality

for i in range(numberOfRounds):
    try:
        # get input from the user and set the context variable
        print("👆 Enter text in the input cell above to chat with the bot. 👆\n")
        sk_input = input(f"{botPrompt} ({(i+1)} of {numberOfRounds})")
        myContext["input"] = sk_input

        # run the chat function
        myResult = await kernel.run_async(chatSkill["Chat"], input_context=myContext)

        # tack onto the history 👇 what's come back from the model
        # ********************************************************
        theNewChatExchange = f"Me: {sk_input}\nAI:{myResult}\n"
        history += theNewChatExchange
        myContext["history"] = history

        # ********************************************************
        # this way the new chat exchange gets passed into the next round

        # announce the number of rounds and the history
        print(f"Chat for {i+1} of {numberOfRounds} rounds with AI:\n{history}")

        # prepare to "prompt" the user with the bot's response
        botPrompt = f"AI: {myResult}"
    except:
        # if the user hits "Escape" we end the chat early
        print("AI: Thanks for the wonderful chat!")
        break

# ⏭️ Next Steps

Run through more advanced examples in the notebooks that are available in our GitHub repo at [https://aka.ms/sk/repo](https://aka.ms/sk/repo).

[Want to deepen your "prompt engineer" chops? Go deeper on semantic AI 🧂🔥! ](../e8-bonus-prompts/notebook.ipynb)

Or stay a longer while and modify the `numberOfRounds` parameter for a longer chat. Even better, chang ethe `personality` parameter in Step 2.2